# Day 3, after lunch: Gridded data analysis and plotting
Focus: netCDF files. Very useful standard, because
 * Provide meta information
 * Provide structure (dimensions versus variables)

We will work with temperatures at 850 hPa from the ERA5 reanalysis; (c) ECMWF. Data accessed through the Copernicus Climate Data Store (2020) and licensed under the Licence to use Copernicus Products.

## Raw access to netCDF

Using the ``netCDF4`` library.

In [ ]:
import netCDF4 as nc
f = nc.Dataset('D3S2/e5.ans.2010072906.850.t.nc', 'r')

So far, so easy. But what's in this file?

In [ ]:
print('Variables:', f.variables)
print('Attributes:', f.ncattrs())

Retrieving variables is straight-forward, but one has to be aware of a subtlety:

In [ ]:
temp = f.variables['t']
# Retrieving the variable object versus the data
print(type(temp), type(temp[:]))

There is a difference between the variable object ``temp`` and the ``numpy.ndarray`` that contains the actual data! The variable object contains all meta-data while the ``numpy.ndarray`` does not.

In [ ]:
temp = temp[:]
print(temp.shape, temp.min(), temp.max())

This raw access is not super difficult or cumbersome, but may be there is an even simpler way?

## Access through ``xarray``

The xarray package is very convenient for working with netCDF data, because it merges the variable object and the data array in one structure. 

In [ ]:
import xarray as xr
ds = xr.open_dataset('D3S2/e5.ans.2010072906.850.t.nc')
#ds
temp_var = ds.variables['t']
t_da = ds['t']
print(type(t_da),type(temp_var))
print(t_da)

The access pattern is very much the same as through the ``netCDF4`` module. However, there is the additional option of accessing ``f['t']`` which contains the both the data and the metadata and can directly be used for analysis and calculations.

For example, the ``DataArray`` allows array indexing through dimension values rather than grid point indexes.

In [ ]:
print(t_da.sel(latitude=60.5))
print(t_da.sel(latitude=60.5, longitude=5.5))

The results of these indexing operations are again of type ``DataArray``, and can hence be further manipulated and analysed.

In addition, they contain up-to-date metadata, so they can directly be saved to netCDF files.

In [ ]:
from datetime import datetime as dt
tosave = t_da.sel(latitude=slice(90,60)) # slightly wierd slice because latitudes are decreasing in the netCDF coordinate
tosave.to_netcdf('example.nc')
!ls -l example.nc

If you have ncdump available, you can also have a direct look at what we saved, and to verify that all metadata has automatically been transferred (and even adapted!) to correctly describe the new netCDF file.

In [ ]:
# Won't work here, because ncdump is only available with another module
# So may be open a second terminal, ssh into cyclone, module load ncview 
# and have a look with ncview and ncdump at the example.nc you created
!ncdump -h example.nc

## Plotting maps

Now that we found an easy way to read and analyse gridded data, let's see how to visualise it. One of the most common types of visualisation is through maps. To plot data on a map, we use the ``cartopy`` module, which is based on ``matplotlib``.

So let's see how to set up a map to plot on.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# Create a matplotlib axes using the Lambert Conic Conformal projection
proj = ccrs.LambertConformal(
    central_longitude=-30, 
    central_latitude=58, 
    standard_parallels=(58, 58)
)
ax = plt.axes(projection=proj)

# Add coastlines and the lat-lon grid
ax.coastlines()
ax.gridlines(ylocs=range(15,76,15))

# set map boundaries by setting the x- and y-limits in meters
#ax.set_xlim([-5.4e6, 5.4e6])
#ax.set_ylim([-3.6e6, 3.6e6])

A lot of text for an empty hull of a map. Now let's fill it with some data. Fortunately, that is very easy, using the standard plotting commands of ``matplotlib``, like ``contourf`` for a filled-contour plot.

In [ ]:
# Explicitly set the Figure size
fig = plt.figure(figsize=(10.0,6.0), dpi=96) 
ax = plt.axes(projection=proj)

# A temperature map
cs = ax.contourf(t_da.longitude, t_da.latitude, t_da, 
            range(268,307,2),
            cmap='RdBu_r',
            transform=ccrs.PlateCarree(), # the projection of the data grid, here lat/lon, i.e. PlateCarree
)
ax.coastlines()
ax.gridlines(ylocs=range(15,76,15))

ax.set_xlim([-5.4e6, 5.4e6])
ax.set_ylim([-3.6e6, 3.6e6])

plt.colorbar(cs, fraction=0.1, shrink=0.9)

Plotting data with many contours can be painfully slow with cartopy. This seems to be the case because of MANY back and forth transformations between the original and the map coordinates. 

One way to circumvent that is to use pcolormesh instead of contourf, in particular while developing the plot.

In [ ]:
# Explicitly set the Figure size
fig = plt.figure(figsize=(10.0,6.0), dpi=96) 
ax = plt.axes(projection=proj)

# A temperature map
cs = ax.pcolormesh(t_da.longitude, t_da.latitude, t_da, 
            vmin=268, vmax=306,
            cmap='RdBu_r',
            transform=ccrs.PlateCarree(), # the projection of the data grid, here lat/lon, i.e. PlateCarree
)
ax.coastlines()
ax.gridlines(ylocs=range(15,76,15))

ax.set_xlim([-5.4e6, 5.4e6])
ax.set_ylim([-3.6e6, 3.6e6])

plt.colorbar(cs, fraction=0.1, shrink=0.9)

Another way to circumvent that is to manually do the projections before, and do the contourf in map coordinates exclusively.

So let's see how to do the projections manually.

In [ ]:
import numpy as np

# Precalculate the x and y positions on the map for the lons and lats of the grid
lons, lats = np.meshgrid(t_da.longitude, t_da.latitude)
coords = proj.transform_points(ccrs.PlateCarree(), lons, lats)
x = coords[:,:,0]
y = coords[:,:,1]

With that in place, onto the plotting.

In [ ]:
# Explicitly set the Figure size
fig = plt.figure(figsize=(10.0,6.0), dpi=96) 

# No more projection here, we already know the x and y positions of the grid points.
# But make sure the aspect ratio is equal. 
ax = plt.axes(aspect='equal') 

# A temperature map, plotted without any explicit projection
cs = ax.contourf(x, y, t_da, np.arange(268,307,2), cmap='RdBu_r')
ax.set_xlim([-5.4e6, 5.4e6])
ax.set_ylim([-3.6e6, 3.6e6])
ax.set_xticks([])
ax.set_yticks([])
plt.colorbar(cs, fraction=0.1, shrink=0.9)

# Manually creating a second pair of axes, sharing the position with one use for the contourf, 
# to serve as the basis for grid/coastlines
axm = fig.add_axes(ax.get_position(), projection=proj, sharex=ax, sharey=ax, frameon=False)
axm.gridlines(ylocs=range(15,76,15))
axm.coastlines()

# Super-fast, but not quite the intended way to be used! So might come with unexpected caveats.

That, hopefully, went a lot faster than the original plot.

Then back to where we left off!

This temperature data is from the peak of the Russian heat wave in 2010, so let's shift the map focus towards Eurasia, and mark some cities on the map. We'll use the cities where we have temperature time series available.

For now we're only interested in the location of these cities (given in the meta data of the netCDF file), but later we'll also include the temperature time series in the plot.

As a first step, we'll have a brief exploratory look at the data. It's also netCDF files, so let's use ``xarray`` again.

In [ ]:
ds = xr.open_dataset('D3S2/e5.ans.2010.850.Moscow.nc')
ds

So, we got three four-dimensional arrays with the dimensions time, level, latitude and longitude. All dimensions except time are of length 1, so we essentially have three time series from one location. Let's have a quick look at the temperature data before we continue.

In [ ]:
print(ds['t'].dims)
print(ds['latitude'])
print(ds['latitude'].values)
ds['t'].plot()

Now that we got a first impression of the data, let's put the cities on the map.

## Exercise 1: Plotting city locations on the map

 * Adapt the map domain to show Eurasia rather than the North Atlantic
 * Take the location information for the three cities Moscow, St. Petersburg and Nizhny Novgorod and mark the respective locations on the map in different colors.

In [ ]:
fig = plt.figure(figsize=(10.0,6.0), dpi=96) 

# new projection, centred roughly on Moscow
proj = ccrs.LambertConformal(
    central_longitude=30, 
    central_latitude=55, 
    standard_parallels=(55, 55)
)

# Plot map
ax = plt.axes(projection=proj)
cs = ax.pcolormesh(t_da.longitude, t_da.latitude, t_da, 
            vmin=268, vmax=306,
            cmap='RdBu_r',
            transform=ccrs.PlateCarree(),
)
ax.coastlines()
ax.gridlines(ylocs=range(15,76,15))

ax.set_xlim([-3.2e6, 3.2e6])
ax.set_ylim([-2.4e6, 2.4e6])
plt.colorbar(cs, fraction=0.1, shrink=0.9)


# obs and info from different places
stations = ['StPeter', 'Moscow', 'NizNovg']

# Add locations of stations on the map
for sidx, station in zip(range(len(stations)), stations):
    ds = xr.open_dataset('D3S2/e5.ans.2010.850.%s.nc' % station)
    lat = ds['latitude'].values
    lon = ds['longitude'].values
    ax.scatter(lon, lat, s=100, facecolor='C%d' % sidx, edgecolor='w', 
               linewidth=1, transform=ccrs.PlateCarree())

plt.savefig('D3S2/ex1.pdf')

## Final plot exercise: Fancy plot with subpanels 

Finally, extend the plot by adding panels showing the temperature time series at these three locations.

 * Set up a 4x3 matrix of subplots, using the entire 3x3 square on the left for the map, and the 1x3 column on the right for the three temperature time series. This kind of panel setup can be achieved through the ``gridspec`` module of ``matplotlib``.
 * Add annotations to each panel to give a brief description of what is shown. ``plt.text`` provides the required functionality.
 * Match the colors of the temperature time series to the color of the markers on the map. 
 * You can add a title for the entire figure rather than the individual panels by ``matplotlib.figure.suptitle``. 

Don't worry if you do not finish this exercise during the session, and feel very much free to experiment making your own kind of fancy plot!

In [ ]:
import numpy as np
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
from datetime import timedelta as td

# width with cb: 6.48 / 0.9 = 7.2, leaving 2.8 for the temperature curves
fig = plt.figure(figsize=(10.0,4.32), dpi=96)

# Setup panels
gs = gridspec.GridSpec(3,4) # set up a 3-by-4 grid of subpanels
gs.update(hspace=0.0, wspace=0.0) # no space between subpanels

# Plot map
axm = plt.subplot(gs[:,:-1], projection=proj) # Map axes covers everything but the rightmost column
cs = axm.pcolormesh(t_da.longitude, t_da.latitude, t_da, 
            vmin=268, vmax=306,
            cmap='RdBu_r',
            transform=ccrs.PlateCarree(), # the projection of the data grid, here lat/lon, i.e. PlateCarree
)
axm.coastlines()
axm.gridlines(ylocs=range(15,76,15))

axm.set_xlim([-3.2e6, 3.2e6])
axm.set_ylim([-2.4e6, 2.4e6])
plt.colorbar(cs, fraction=0.1, shrink=0.9)

showdate = dt(2010,7,29,6)
datestr = showdate.strftime('%Y-%m-%d, %H UTC')
axm.text(0.05, 0.05, datestr, transform=axm.transAxes, 
         bbox=dict(boxstyle="round", fc='w', alpha=0.7))

# Plot temperature time series, 3 weeks before and after the peak of the heat wave
stationnames = ['St. Petersburg', 'Moscow', 'Nizhny Novgorod']
for sidx, station, name in zip(range(len(stations)), stations, stationnames):
    color = 'C%d' % sidx
    ds = xr.open_dataset('D3S2/e5.ans.2010.850.%s.nc' % station)
    ds = ds.sel(time=slice(showdate - td(21), showdate + td(21))) # select +/- 21 days around the showdate
    T = ds['t']

    lat = ds['latitude'].values
    lon = ds['longitude'].values
    axm.scatter(lon, lat, s=100, facecolor=color, edgecolor='w', 
               linewidth=1, transform=ccrs.PlateCarree())    
    
    ax = plt.subplot(gs[sidx,-1])
    ax.plot([showdate, showdate], [278,296], color='0.2', linestyle='--') # vertical line
    #T.plot(color=color) # <- quick, but less easy to customise 
    ax.plot(T.time, T, color)
    ax.yaxis.tick_right()
    
    if station == stations[-1]:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d'))
        ax.set_xlabel('Day of Jul/Aug 2010')
    else:
        ax.set_xticks([])
    
    ax.text(0.05, 0.05, name, color=color, transform=ax.transAxes)

# After all this work, finally, a title for the entire figure.
fig.suptitle('850 hPa Temperatures [K]')
plt.savefig('D3S2/ex2.pdf')